Reading JSON data of an existing coupled system

In [1]:
import os
import json


input_path = os.path.join(
    os.path.dirname(os.path.abspath('')),
    'mixed_fid_models/',
    'ProjectParametersCoSimFSI.json')

with open(input_path) as f:
    data = json.load(f)

data

{'problem_data': {'start_time': 0.0,
  'end_time': 1.0,
  'echo_level': 0,
  'print_colors': True,
  'parallel_type': 'OpenMP'},
 'solver_settings': {'type': 'coupled_solvers.gauss_seidel_strong',
  'num_coupling_iterations': 20,
  'echo_level': 3,
  'data_transfer_operators': {'direct_transfer': {'type': 'kratos_mapping',
    'mapper_settings': {'mapper_type': 'nearest_neighbor', 'echo_level': 3}}},
  'predictors': [],
  'convergence_accelerators': [{'type': 'aitken',
    'solver': 'low_fid_fluid',
    'data_name': 'pitch_angle'}],
  'convergence_criteria': [{'type': 'relative_norm_previous_residual',
    'solver': 'low_fid_fluid',
    'data_name': 'pitch_angle',
    'abs_tolerance': 1e-07,
    'rel_tolerance': 1e-05}],
  'coupling_sequence': [{'name': 'low_fid_fluid',
    'output_data_list': [],
    'input_data_list': []},
   {'name': 'spring_structure',
    'input_data_list': [{'data': 'lift_force',
      'from_solver': 'low_fid_fluid',
      'from_solver_data': 'lift_force',
      

Connecting to the knowledge base

In [2]:
from coupled_modeling import KnowledgeBase

kb = KnowledgeBase('http://127.0.0.1:5000/api/v1.0/')

kb

Importing data of the existing coupled system

In [3]:
mixed_fid_models = kb.import_coupled_kratos('mixed_fid_models', data)

mixed_fid_models

'instance_104'

Creating a new coupled system

In [4]:
Onera_FSI = kb.create_coupled('Onera_FSI')

Onera_FSI

'instance_125'

Checking properties of the imported coupled system

In [5]:
mixed_fid_models_props = kb.get_instance_properties(mixed_fid_models)

mixed_fid_models_props

{'label': 'mixed_fid_models',
 'problem_data': 'instance_105',
 'solver_settings': 'instance_106'}

Checking problem data of the imported coupled system

In [6]:
problem_data_old = mixed_fid_models_props['problem_data']
kb.get_instance_properties(problem_data_old)

{'echo_level': 0,
 'end_time': 1.0,
 'parallel_type': 'OpenMP',
 'print_colors': True,
 'start_time': 0.0}

All properties of the existing problem data instance can be reused, except of echo_level. Copying problem data from the existing coupled system with a new echo_level value

In [7]:
problem_data_new = kb.copy_instance(
    problem_data_old,
    Onera_FSI,
    {'echo_level': 2})

problem_data_new

'instance_126'

Getting solver settings of the imported coupled system

In [8]:
solver_settings_old = mixed_fid_models_props['solver_settings']
solver_settings_old_props = kb.get_instance_properties(solver_settings_old)

solver_settings_old_props

{'convergence_accelerators': ['instance_109'],
 'convergence_criteria': ['instance_110'],
 'coupling_sequence': ['instance_111', 'instance_112'],
 'data_transfer_operators': 'instance_107',
 'echo_level': 3,
 'num_coupling_iterations': 20,
 'solvers': ['instance_115', 'instance_120'],
 'type': 'coupled_solvers.gauss_seidel_strong'}

The copying method does not copy nested instances. Thus, we can copy the solver settings from the existing coupled system with the old *num_coupling_iterations* and *type*, but with a new *echo_level*

In [9]:
solver_settings_new = kb.copy_instance(
    solver_settings_old,
    Onera_FSI,
    {'echo_level': 4})

solver_settings_new

'instance_127'

Getting existing convergence accelerators

In [10]:
convergence_accelerators_old = solver_settings_old_props['convergence_accelerators'][0]
convergence_accelerators_old_props = kb.get_instance_properties(convergence_accelerators_old)

convergence_accelerators_old_props

{'data_name': 'pitch_angle', 'solver': 'low_fid_fluid', 'type': 'aitken'}

Only *type* can be reused, thus, we create new convergence accelerators for the new coupled system from scratch, passing the class, the parent instance and new property values

In [11]:
convergence_accelerators_new = kb.create_instance(
    'convergence_accelerators',
    solver_settings_new,
    {'data_name': 'displacements', 'solver': 'CFD', 'type': 'aitken'})

convergence_accelerators_new

'instance_128'

Checking existing convergence criteria

In [12]:
convergence_criteria_old = solver_settings_old_props['convergence_criteria'][0]
kb.get_instance_properties(convergence_criteria_old)

{'abs_tolerance': 1e-07,
 'data_name': 'pitch_angle',
 'rel_tolerance': 1e-05,
 'solver': 'low_fid_fluid',
 'type': 'relative_norm_previous_residual'}

Copying convergence criteria, passing new *data_name*, *solver* and *type*

In [13]:
convergence_criteria_new = kb.copy_instance(convergence_criteria_old, solver_settings_new, {
    'data_name': 'displacements',
    'solver': 'CFD',
    'type': 'relative_norm_initial_residual'})

convergence_criteria_new

'instance_129'

Checking existing coupling sequences

In [14]:
coupling_sequence_old_0_props = kb.get_instance_properties(solver_settings_old_props['coupling_sequence'][0])

coupling_sequence_old_0_props

{'name': 'low_fid_fluid'}

In [15]:
coupling_sequence_old_1_props = kb.get_instance_properties(solver_settings_old_props['coupling_sequence'][1])

coupling_sequence_old_1_props

{'input_data_list': ['instance_113'],
 'name': 'spring_structure',
 'output_data_list': ['instance_114']}

Checking existing input data list

In [16]:
input_data_list_old = coupling_sequence_old_1_props['input_data_list'][0]
kb.get_instance_properties(input_data_list_old)

{'data': 'lift_force',
 'data_transfer_operator': 'direct_transfer',
 'from_solver': 'low_fid_fluid',
 'from_solver_data': 'lift_force'}

Checking existing output data list

In [17]:
output_data_list_old = coupling_sequence_old_1_props['output_data_list'][0]
kb.get_instance_properties(output_data_list_old)

{'data': 'pitch_angle',
 'data_transfer_operator': 'direct_transfer',
 'to_solver': 'low_fid_fluid',
 'to_solver_data': 'pitch_angle'}

We create a new coupling sequence by creating its elements one by one

In [18]:
coupling_sequence_new_0 = kb.create_instance('coupling_sequence', solver_settings_new, {
    'name': 'CFD'})

coupling_sequence_new_0

'instance_130'

Creating another coupling sequence

In [19]:
coupling_sequence_new_1 = kb.create_instance('coupling_sequence', solver_settings_new, {
    'name': 'SM'})

coupling_sequence_new_1

'instance_131'

Then we create new nested instances for the second item in the new coupling sequence. Copying a new input data list

In [20]:
input_data_list_new = kb.copy_instance(input_data_list_old, coupling_sequence_new_1, {
    'data_transfer_operator': 'mapping_operation',
    'from_solver': 'CFD',
    'data_transfer_operator_options': 'use_transpose'})

input_data_list_new

'instance_132'

Creating a new output data list

In [21]:
output_data_list_new = kb.create_instance('output_data_list', coupling_sequence_new_1, {
    'data': 'displacements',
    'data_transfer_operator': 'mapping_operation',
    'to_solver': 'CFD',
    'to_solver_data': 'displacements'})

output_data_list_new

'instance_135'

Checking existing data transfer operators

In [22]:
data_transfer_operators_old = solver_settings_old_props['data_transfer_operators']
data_transfer_operators_old_props = kb.get_instance_properties(data_transfer_operators_old)

data_transfer_operators_old_props

{'label': 'direct_transfer',
 'mapper_settings': 'instance_108',
 'type': 'kratos_mapping'}

Checking existing mapper settings

In [23]:
direct_transfer = data_transfer_operators_old_props['mapper_settings']
kb.get_instance_properties(direct_transfer)

{'echo_level': 3, 'mapper_type': 'nearest_neighbor'}

Creating a mapping operation instance

In [24]:
mapping_operation = kb.create_instance('data_transfer_operators', solver_settings_new, {
    'label': 'mapping_operation',
    'type': 'kratos_mapping'})

mapping_operation

'instance_136'

Creating new mapper settings

In [25]:
mapper_settings = kb.create_instance('mapper_settings', mapping_operation, {
    'mapper_type': 'nearest_neighbor',
    'use_initial_configuration': True})

mapper_settings

'instance_137'

Checking the first existing solver

In [26]:
solver_old_0 = solver_settings_old_props['solvers'][0]
solver_old_0_props = kb.get_instance_properties(solver_old_0)

solver_old_0_props

{'data': ['instance_118', 'instance_119'],
 'io_settings': 'instance_117',
 'label': 'low_fid_fluid',
 'solver_wrapper_settings': 'instance_116',
 'type': 'solver_wrappers.external.remote_controlled_solver_wrapper'}

Creating an instance of the CFD solver by copying the first solver

In [27]:
CFD = kb.copy_instance(solver_old_0, solver_settings_new, {
    'label': 'CFD'})

CFD

'instance_138'

Checking existing data

In [28]:
kb.get_instance_properties(solver_old_0_props['data'][0])

{'label': 'pitch_angle',
 'location': 'node_historical',
 'model_part_name': 'single_node_mesh_f',
 'variable_name': 'SCALAR_DISPLACEMENT'}

In [29]:
kb.get_instance_properties(solver_old_0_props['data'][1])

{'label': 'lift_force',
 'location': 'node_historical',
 'model_part_name': 'single_node_mesh_f',
 'variable_name': 'SCALAR_FORCE'}

Creating displacements data for the CFD solver instance

In [30]:
displacements_0 = kb.create_instance('data', CFD, {
    'label': 'displacements',
    'dimensions': 3,
    'model_part_name': 'WING',
    'variable_name': 'MESH_DISPLACEMENT'})

displacements_0

'instance_139'

Creating lift force data instance for the CFD solver

In [31]:
lift_force_0 = kb.create_instance('data', CFD, {
    'label': 'lift_force',
    'dimensions': 3,
    'model_part_name': 'WING',
    'variable_name': 'REACTION'})

lift_force_0

'instance_142'

Checking existing IO settings

In [32]:
io_settings_old = solver_old_0_props['io_settings']
kb.get_instance_properties(io_settings_old)

{'communication_format': 'file',
 'connect_to': 'run_fluid',
 'echo_level': 4,
 'type': 'kratos_co_sim_io'}

Copying io settings

In [33]:
io_settings_new = kb.copy_instance(io_settings_old, CFD, {
    'connect_to': 'run_SU2'})

io_settings_new

'instance_144'

Checking existing solver wrapper settings

In [34]:
solver_wrapper_settings_old_0 = solver_old_0_props['solver_wrapper_settings']
kb.get_instance_properties(solver_wrapper_settings_old_0)

{'export_data': ['pitch_angle'],
 'import_data': ['lift_force'],
 'import_meshes': ['single_node_mesh_f']}

Copying solver wrapper settings

In [35]:
solver_wrapper_settings_new_0 = kb.copy_instance(solver_wrapper_settings_old_0, CFD, {
    'export_data': 'displacements',
    'import_meshes': 'WING'})

solver_wrapper_settings_new_0

'instance_146'

Checking another existing solver

In [36]:
solver_old_1 = solver_settings_old_props['solvers'][1]
solver_old_1_props = kb.get_instance_properties(solver_old_1)

solver_old_1_props

{'data': ['instance_123', 'instance_124'],
 'io_settings': 'instance_122',
 'label': 'spring_structure',
 'solver_wrapper_settings': 'instance_121',
 'type': 'solver_wrappers.external.remote_controlled_solver_wrapper'}

Creating an instance for the SM solver

In [37]:
SM = kb.create_instance('solvers', solver_settings_new, {
    'label': 'SM',
    'type': 'solver_wrappers.kratos.structural_mechanics_wrapper'})

SM

'instance_147'

Checking existing data of the second solver

In [38]:
kb.get_instance_properties(solver_old_1_props['data'][0])

{'label': 'pitch_angle',
 'location': 'node_historical',
 'model_part_name': 'single_node_mesh_s',
 'variable_name': 'SCALAR_DISPLACEMENT'}

In [49]:
kb.get_instance_properties(solver_old_1_props['data'][1])

{'label': 'lift_force',
 'location': 'node_historical',
 'model_part_name': 'single_node_mesh_s',
 'variable_name': 'SCALAR_FORCE'}

Coping data for the second solver from the first one

In [40]:
displacements_1 = kb.copy_instance(displacements_0, SM, {
    'model_part_name': 'Structure.interface',
    'variable_name': 'DISPLACEMENT'})

displacements_1

'instance_149'

Copying another data from the first solver

In [41]:
lift_force_1 = kb.copy_instance(lift_force_0, SM, {
    'model_part_name': 'Structure.interface',
    'variable_name': 'POINT_LOAD'})

lift_force_1

'instance_152'

Checking wrapper settings of the second existing solver

In [42]:
solver_wrapper_settings_old_1 = solver_old_1_props['solver_wrapper_settings']
kb.get_instance_properties(solver_wrapper_settings_old_1)

{'export_data': ['lift_force'],
 'import_data': ['pitch_angle'],
 'import_meshes': ['single_node_mesh_s']}

Creating solver wrapper settings for the second new solver

In [43]:
solver_wrapper_settings_new_1 = kb.create_instance('solver_wrapper_settings', SM, {
    'input_file': 'ProjectParametersSM'})

solver_wrapper_settings_new_1

'instance_154'

Exporting created coupled system in JSON

In [44]:
export = kb.export_coupled_kratos(Onera_FSI)

export

{'problem_data': {'echo_level': 2,
  'end_time': 1.0,
  'parallel_type': 'OpenMP',
  'print_colors': True,
  'start_time': 0.0},
 'solver_settings': {'convergence_accelerators': [{'data_name': 'displacements',
    'solver': 'CFD',
    'type': 'aitken'}],
  'convergence_criteria': [{'abs_tolerance': 1e-07,
    'data_name': 'displacements',
    'rel_tolerance': 1e-05,
    'solver': 'CFD',
    'type': 'relative_norm_initial_residual'}],
  'coupling_sequence': [{'name': 'CFD'},
   {'input_data_list': [{'data': 'lift_force',
      'data_transfer_operator': 'mapping_operation',
      'data_transfer_operator_options': ['use_transpose'],
      'from_solver': 'CFD',
      'from_solver_data': 'lift_force'}],
    'name': 'SM',
    'output_data_list': [{'data': 'displacements',
      'data_transfer_operator': 'mapping_operation',
      'to_solver': 'CFD',
      'to_solver_data': 'displacements'}]}],
  'data_transfer_operators': {'mapping_operation': {'mapper_settings': {'mapper_type': 'nearest_nei

Saving exported data into a file

In [45]:
export_path = os.path.join(
    os.path.abspath(''),
    'export_onera_fsi.json')

with open(export_path, 'w') as file:
    json.dump(export, file, indent=2)

Inferring new classes from the created coupled system

In [46]:
kb.infer_coupled_structure(Onera_FSI)

''

Updating the knowledge base

In [47]:
kb.save()

''

Getting a local copy of the knowledge base

In [48]:
onto_path = os.path.join(
    os.path.abspath(''),
    'demo_client.owl')

kb.save_locally(onto_path)